
**Opening a coffe shop in Sumaré, Brazil**
- Build a dataframe of neighborhoods in Sumaré, Brazil from wikpedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new coffee shop
***


In [10]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json
from geopy.geocoders import Nominatim 
import geocoder
import requests
from bs4 import BeautifulSoup 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print("Libraries imported.")

Libraries imported.


### Get data from Wikipedia

In [3]:
data = requests.get("https://pt.wikipedia.org/wiki/Lista_de_bairros_de_Sumaré").text

In [4]:
soup = BeautifulSoup(data, 'html.parser')

In [5]:
neighborhoodList = []

In [6]:
for row in soup.find_all("div", class_="mw-parser-output")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [28]:
sumare_df = pd.DataFrame({"Neighborhood": neighborhoodList})
sumare_df.head(188)

,Neighborhood
0,Chácaras Cruzeiro do Sul - AR7 – Zona Rural
1,Chácaras de Recreio Estrela D'Alva - AR7 – Zon...
2,Chácaras Monte Alegre – AR6 – Picerno
3,Chácaras Primavera - AR7 – Zona Rural
4,Estância Árvore da Vida - AR7 – Zona Rural
5,Haras Larissa – Sítio Santo Antônio (porção em...
6,Jardim Alvorada - AR1 – Zona Central
7,Jardim Basilicata - AR6 – Picerno
8,Jardim Basso - AR1 – Zona Central
9,Jardim Bela Vista - AR2 – Nova Veneza


In [29]:
sumare_df.shape

(189, 1)

### Get the Geo Location

In [30]:
def get_latlng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Sumaré, Brazil'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [31]:
coords = [ get_latlng(neighborhood) for neighborhood in sumare_df["Neighborhood"].tolist() ]

In [32]:
coords

[[-23.511539999999968, -46.532479999999964],
 [-22.825879999999927, -47.269199999999955],
 [-22.806389999999965, -47.24729999999994],
 [-23.237199999999973, -47.29900999999995],
 [-22.825879999999927, -47.269199999999955],
 [-22.825879999999927, -47.269199999999955],
 [-20.572059999999965, -47.390529999999956],
 [-22.818289999999934, -47.26347999999996],
 [-23.42570999999998, -51.938219999999944],
 [-22.81454999999994, -47.239439999999945],
 [-20.522279999999967, -47.387279999999976],
 [-23.42570999999998, -51.938219999999944],
 [-22.818289999999934, -47.26347999999996],
 [-11.149999999999977, -42.049999999999955],
 [-27.624119999999948, -48.663629999999955],
 [-23.42570999999998, -51.938219999999944],
 [-22.125089999999943, -51.410499999999956],
 [-22.818289999999934, -47.26347999999996],
 [-19.467129220432202, -42.517233582087385],
 [-23.42570999999998, -51.938219999999944],
 [-22.007709999999975, -47.88848999999993],
 [-23.42570999999998, -51.938219999999944],
 [-22.81454999999994, 

In [33]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [34]:
sumare_df['Latitude'] = df_coords['Latitude']
sumare_df['Longitude'] = df_coords['Longitude']

In [35]:
# check the neighborhoods and the coordinates
print(sumare_df.shape)
sumare_df

(189, 3)


,Neighborhood,Latitude,Longitude
0,Chácaras Cruzeiro do Sul - AR7 – Zona Rural,-23.511540,-46.532480
1,Chácaras de Recreio Estrela D'Alva - AR7 – Zon...,-22.825880,-47.269200
2,Chácaras Monte Alegre – AR6 – Picerno,-22.806390,-47.247300
3,Chácaras Primavera - AR7 – Zona Rural,-23.237200,-47.299010
4,Estância Árvore da Vida - AR7 – Zona Rural,-22.825880,-47.269200
5,Haras Larissa – Sítio Santo Antônio (porção em...,-22.825880,-47.269200
6,Jardim Alvorada - AR1 – Zona Central,-20.572060,-47.390530
7,Jardim Basilicata - AR6 – Picerno,-22.818290,-47.263480
8,Jardim Basso - AR1 – Zona Central,-23.425710,-51.938220
9,Jardim Bela Vista - AR2 – Nova Veneza,-22.814550,-47.239440


In [36]:
# save the DataFrame as CSV file
sumare_df.to_csv("sumare_df.csv", index=False)

### Create a map of Sumaré with neighborhoods

In [37]:
address = 'Sumaré, Brazil'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Sumaré, Brazil {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Sumaré, Brazil -22.8217964, -47.267105.


In [38]:
map_sumare = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, neighborhood in zip(sumare_df['Latitude'], sumare_df['Longitude'], sumare_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_sumare)  
    
map_sumare

In [39]:
map_sumare.save('map_sumare.html')

### Foursquare API to explore the neighborhoods

In [40]:
CLIENT_ID = 'ZS440BAXUPK342YI2ZPBWURUYVATI2SVXYIL5PGWGPK32EUP' 
CLIENT_SECRET = 'APOI1RRH5P0ATD2N2PP1SKRRPT31MPDWWI15FGNNWSU3KB4C' 
VERSION = '20180605'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZS440BAXUPK342YI2ZPBWURUYVATI2SVXYIL5PGWGPK32EUP
CLIENT_SECRET:APOI1RRH5P0ATD2N2PP1SKRRPT31MPDWWI15FGNNWSU3KB4C


### Let's get the top 80 venues that are with a radius of 1500m.

In [41]:
radius = 1500
LIMIT = 80

venues = []

for lat, long, neighborhood in zip(sumare_df['Latitude'], sumare_df['Longitude'], sumare_df['Neighborhood']):
    

    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    

    results = requests.get(url).json()["response"]['groups'][0]['items']
    

    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [42]:

venues_df = pd.DataFrame(venues)


venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6648, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Chácaras Cruzeiro do Sul - AR7 – Zona Rural,-23.51154,-46.53248,Sodiê Doces,-23.506624,-46.530876,Café
1,Chácaras Cruzeiro do Sul - AR7 – Zona Rural,-23.51154,-46.53248,Por Uma America Unida - Moto Bar,-23.515586,-46.531676,Bar
2,Chácaras Cruzeiro do Sul - AR7 – Zona Rural,-23.51154,-46.53248,Reduto da Esfiha,-23.515639,-46.527972,Mediterranean Restaurant
3,Chácaras Cruzeiro do Sul - AR7 – Zona Rural,-23.51154,-46.53248,Russo Academia,-23.516774,-46.530740,Gymnastics Gym
4,Chácaras Cruzeiro do Sul - AR7 – Zona Rural,-23.51154,-46.53248,Padaria Anjo da Guarda,-23.505868,-46.529179,Bakery


## Check how many venues returned for each neighborhood

In [43]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Chácaras Bela Vista - AR2 – Nova Veneza,30,30,30,30,30,30
Chácaras Cruzeiro do Sul - AR7 – Zona Rural,79,79,79,79,79,79
Chácaras Monte Alegre – AR6 – Picerno,17,17,17,17,17,17
Chácaras Nova Veneza - AR2 – Nova Veneza,31,31,31,31,31,31
Chácaras Novo Horizonte e San Martin - AR3 – Matão,10,10,10,10,10,10
Chácaras Primavera - AR7 – Zona Rural,4,4,4,4,4,4
Chácaras Reunidas Anhanguera - AR2 – Nova Veneza,31,31,31,31,31,31
Chácaras Santa Antonieta - AR2 – Nova Veneza,31,31,31,31,31,31
Chácaras de Recreio Estrela D'Alva - AR7 – Zona Rural,74,74,74,74,74,74


## Find out how many unique categories

In [44]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 189 uniques categories.


In [45]:
venues_df['VenueCategory'].unique()[:50]

array(['Café', 'Bar', 'Mediterranean Restaurant', 'Gymnastics Gym',
       'Bakery', 'Burger Joint', 'Seafood Restaurant', 'Park',
       'Churrascaria', 'Brazilian Restaurant', 'Hookah Bar',
       'Northeastern Brazilian Restaurant', 'Beer Garden', 'Plaza',
       'Pizza Place', 'Juice Bar', 'Grocery Store', 'Dessert Shop',
       'Supermarket', 'Pharmacy', 'Farmers Market', 'Pet Store',
       'Fast Food Restaurant', 'Sandwich Place', 'Skate Park',
       'Restaurant', 'Track', 'General Entertainment',
       'Japanese Restaurant', 'Chocolate Shop', 'Coffee Shop',
       'Gym / Fitness Center', 'Gym', 'Italian Restaurant',
       'Warehouse Store', 'Nightclub', 'Arts & Entertainment',
       'Train Station', 'Shopping Mall', 'Jewelry Store',
       'Chinese Restaurant', 'Electronics Store', 'River', 'Soccer Field',
       'Pier', 'Ice Cream Shop', 'BBQ Joint', 'Pet Service', 'Soup Place',
       'Arts & Crafts Store'], dtype=object)

In [46]:
"Coffee Shop" in venues_df['VenueCategory'].unique()

True

## Analyze Each Neighborhood

In [47]:
sumare_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

sumare_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

fixed_columns = [sumare_onehot.columns[-1]] + list(sumare_onehot.columns[:-1])
sumare_onehot = sumare_onehot[fixed_columns]

print(sumare_onehot.shape)
sumare_onehot.head()

(6648, 190)


,Neighborhoods,Acai House,Airport,Airport Lounge,American Restaurant,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Business Service,Café,Candy Store,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Dumpling Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Historic Site,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Lake,Laser Tag,Liquor Store,Lottery Retailer,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mineiro Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Motel,Mountain,Multiplex,Music Venue,Nightclub,Noodle House,Northeastern Brazilian Restaurant,Nudist Beach,Office,Optical Shop,Other Great Outdoors,Paper / Office Supplies Store,Park,Pastelaria,Pastry Shop,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Racetrack,Recreation Center,Rental Car Location,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Tapiocaria,Tea Room,Tennis Stadium,Theater,Toy / Game Store,Track,Trail,Train Station,Turkish Home Cooking Restaurant,Vegetarian / Vegan Restaurant,Video Store,Warehouse,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Chácaras Cruzeiro do Sul - AR7 – Zona Rural,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Chácaras Cruzeiro do Sul - AR7 – Zona Rural,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Chácaras Cruzeiro do Sul - AR7 – Zona Rural,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Chácaras Cruzeiro do Sul - AR7 – Zona Rural,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

## Group rows by neighborhood

In [48]:
sumare_grouped = sumare_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(sumare_grouped.shape)
sumare_grouped

(186, 190)


,Neighborhoods,Acai House,Airport,Airport Lounge,American Restaurant,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Business Service,Café,Candy Store,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Dumpling Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Historic Site,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Lake,Laser Tag,Liquor Store,Lottery Retailer,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mineiro Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Motel,Mountain,Multiplex,Music Venue,Nightclub,Noodle House,Northeastern Brazilian Restaurant,Nudist Beach,Office,Optical Shop,Other Great Outdoors,Paper / Office Supplies Store,Park,Pastelaria,Pastry Shop,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Racetrack,Recreation Center,Rental Car Location,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Tapiocaria,Tea Room,Tennis Stadium,Theater,Toy / Game Store,Track,Trail,Train Station,Turkish Home Cooking Restaurant,Vegetarian / Vegan Restaurant,Video Store,Warehouse,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Chácaras Bela Vista - AR2 – Nova Veneza,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.033333,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.033333,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.033333,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.0000,0.033333,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.0000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.033333,0.000000,0.033333,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.

In [49]:
len(sumare_grouped[sumare_grouped["Coffee Shop"] > 0])

128

## Create a new dataframe for Coffee Shop

In [50]:
sumare_coffee = sumare_grouped[["Neighborhoods","Coffee Shop"]]

In [51]:
sumare_coffee.head(100)

,Neighborhoods,Coffee Shop
0,Chácaras Bela Vista - AR2 – Nova Veneza,0.033333
1,Chácaras Cruzeiro do Sul - AR7 – Zona Rural,0.012658
2,Chácaras Monte Alegre – AR6 – Picerno,0.000000
3,Chácaras Nova Veneza - AR2 – Nova Veneza,0.032258
4,Chácaras Novo Horizonte e San Martin - AR3 – M...,0.100000
5,Chácaras Primavera - AR7 – Zona Rural,0.000000
6,Chácaras Reunidas Anhanguera - AR2 – Nova Veneza,0.032258
7,Chácaras Santa Antonieta - AR2 – Nova Veneza,0.032258
8,Chácaras de Recreio Estrela D'Alva - AR7 – Zon...,0.027027
9,Cidade Nova - AR5 – Maria Antônia,0.000000


## Cluster Neighborhoods


In [52]:
sclusters = 3
sm_clustering = sumare_coffee.drop(["Neighborhoods"], 1)
kmeans = KMeans(n_clusters=sclusters, random_state=0).fit(sm_clustering)
kmeans.labels_[0:10] 

array([0, 2, 2, 0, 1, 2, 0, 0, 0, 2])

In [53]:
sm_merged = sumare_coffee.copy()
sm_merged["Cluster Labels"] = kmeans.labels_

In [54]:
sm_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
sm_merged.head()

,Neighborhood,Coffee Shop,Cluster Labels
0,Chácaras Bela Vista - AR2 – Nova Veneza,0.033333,0
1,Chácaras Cruzeiro do Sul - AR7 – Zona Rural,0.012658,2
2,Chácaras Monte Alegre – AR6 – Picerno,0.000000,2
3,Chácaras Nova Veneza - AR2 – Nova Veneza,0.032258,0
4,Chácaras Novo Horizonte e San Martin - AR3 – M...,0.100000,1


In [55]:
sm_merged = sm_merged.join(sumare_df.set_index("Neighborhood"), on="Neighborhood")


print(sm_merged.shape)
sm_merged.head() # check the last columns!

(187, 5)


,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
0,Chácaras Bela Vista - AR2 – Nova Veneza,0.033333,0,-22.814550,-47.239440
1,Chácaras Cruzeiro do Sul - AR7 – Zona Rural,0.012658,2,-23.511540,-46.532480
2,Chácaras Monte Alegre – AR6 – Picerno,0.000000,2,-22.806390,-47.247300
3,Chácaras Nova Veneza - AR2 – Nova Veneza,0.032258,0,-19.467129,-42.517234
4,Chácaras Novo Horizonte e San Martin - AR3 – M...,0.100000,1,-22.829010,-47.167230


In [56]:
print(sm_merged.shape)
sm_merged.sort_values(["Cluster Labels"], inplace=True)
sm_merged

(187, 5)


,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
0,Chácaras Bela Vista - AR2 – Nova Veneza,0.033333,0,-22.814550,-47.239440
118,Parque Franceschini - AR1 – Zona Central,0.025000,0,-23.425710,-51.938220
116,Parque Emília - AR1 – Zona Central,0.025000,0,-23.425710,-51.938220
115,Parque Dante Marmirolli - AR1 – Zona Central,0.027027,0,-22.825880,-47.269200
113,Loteamento Tranquilo Menuzzo - AR1 – Zona Central,0.027027,0,-22.825880,-47.269200
112,Loteamento Tomás Dedona e Vitório Pasan - AR1 ...,0.027027,0,-22.825880,-47.269200
111,Loteamento Tereza Buchianeri Biancalana - AR1 ...,0.027027,0,-22.825880,-47.269200
108,Loteamento Otávio Brochini - AR1 – Zona Central,0.027027,0,-22.825880,-47.269200
107,Loteamento Oreste Ongaro - AR1 – Zona Central,0.027027,0,-22.825880,-47.269200
106,Loteamento José e Júlio Vasconcellos - AR1 – Z...,0.027027,0,-22.825880,-47.269200


**Finally, let's visualize the resulting clusters**

In [58]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(sclusters)
ys = [i+x+(i*x)**2 for i in range(sclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sm_merged['Latitude'], sm_merged['Longitude'], sm_merged['Neighborhood'], sm_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [59]:
map_clusters.save('map_clusters.html')

## Examine Clusters

#### Cluster 0

In [60]:
sm_merged.loc[sm_merged['Cluster Labels'] == 0]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
0,Chácaras Bela Vista - AR2 – Nova Veneza,0.033333,0,-22.814550,-47.239440
118,Parque Franceschini - AR1 – Zona Central,0.025000,0,-23.425710,-51.938220
116,Parque Emília - AR1 – Zona Central,0.025000,0,-23.425710,-51.938220
115,Parque Dante Marmirolli - AR1 – Zona Central,0.027027,0,-22.825880,-47.269200
113,Loteamento Tranquilo Menuzzo - AR1 – Zona Central,0.027027,0,-22.825880,-47.269200
112,Loteamento Tomás Dedona e Vitório Pasan - AR1 ...,0.027027,0,-22.825880,-47.269200
111,Loteamento Tereza Buchianeri Biancalana - AR1 ...,0.027027,0,-22.825880,-47.269200
108,Loteamento Otávio Brochini - AR1 – Zona Central,0.027027,0,-22.825880,-47.269200
107,Loteamento Oreste Ongaro - AR1 – Zona Central,0.027027,0,-22.825880,-47.269200
106,Loteamento José e Júlio Vasconcellos - AR1 – Z...,0.027027,0,-22.825880,-47.269200


#### Cluster 1

In [61]:
sm_merged.loc[sm_merged['Cluster Labels'] == 1]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
177,Vila San Martin - AR3 – Matão,0.100000,1,-22.829010,-47.167230
155,Residencial Parque Pavan - AR3 – Matão,0.100000,1,-22.829010,-47.167230
34,Jardim Eunice - AR3 – Matão,0.100000,1,-22.829010,-47.167230
114,Nova Terra - AR3 – Matão,0.100000,1,-22.829010,-47.167230
119,Parque General Osório - AR3 – Matão,0.100000,1,-22.829010,-47.167230
4,Chácaras Novo Horizonte e San Martin - AR3 – M...,0.100000,1,-22.829010,-47.167230
128,Parque Progresso II - AR3 – Matão,0.100000,1,-22.829010,-47.167230
127,Parque Progresso - AR3 – Matão,0.100000,1,-22.829010,-47.167230
131,Parque Residencial Fantinatti - AR3 – Matão,0.100000,1,-22.829010,-47.167230
10,Condomínio Coronel - AR3 – Matão,0.100000,1,-22.829010,-47.167230


#### Cluster 2

In [62]:
sm_merged.loc[sm_merged['Cluster Labels'] == 2]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
15,Jardim Aclimação - AR4 – Área Cura,0.000000,2,-22.846590,-47.172010
165,São Judas Tadeu II - AR4 – Área Cura,0.000000,2,-22.853144,-47.163623
16,Jardim Alvorada - AR1 – Zona Central,0.000000,2,-20.572060,-47.390530
80,Jardim São Carlos - AR1 – Zona Central,0.000000,2,-22.979580,-49.846520
161,Residencial Ypiranga - AR4 – Área Cura,0.000000,2,-22.846590,-47.172010
164,São Judas Tadeu - AR4 – Área Cura,0.000000,2,-22.855667,-47.165805
154,Residencial Itália - AR4 – Área Cura,0.000000,2,-22.846590,-47.172010
17,Jardim Amélia - AR5 – Maria Antônia,0.000000,2,-22.811240,-47.211810
158,Residencial Santa Joana - AR4 – Área Cura,0.000000,2,-22.846590,-47.172010
82,Jardim São Francisco - AR4 – Área Cura,0.000000,2,-22.857761,-47.188475
